In [1]:
data.raw = read.table("crabs.dat.txt", header = TRUE)

In [2]:
data.raw$color = factor(data.raw$color)
data.raw$y = factor(data.raw$y)

In [4]:
require(lmtest)

model.all_variables = glm(spine ~ sat + y + weight + width + color, family = quasipoisson(link = log), data = data.raw)
summary(model.all_variables)


Call:
glm(formula = spine ~ sat + y + weight + width + color, family = quasipoisson(link = log), 
    data = data.raw)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2842  -0.2786   0.1515   0.3177   1.3208  

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.009538   0.475969  -0.020    0.984    
sat         -0.007227   0.010783  -0.670    0.504    
y1           0.091759   0.071749   1.279    0.203    
weight      -0.132128   0.087783  -1.505    0.134    
width        0.022427   0.023996   0.935    0.351    
color2       0.581651   0.126059   4.614 7.89e-06 ***
color3       0.716702   0.130640   5.486 1.52e-07 ***
color4       0.766032   0.138544   5.529 1.24e-07 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for quasipoisson family taken to be 0.2365512)

    Null deviance: 56.157  on 172  degrees of freedom
Residual deviance: 44.605  on 165  degrees of freedom
AIC: NA

Number of 

In [6]:
model.color = glm(spine ~ color, family = quasipoisson(link = log), data = data.raw)
model.weight = glm(spine ~ weight, family = quasipoisson(link = log), data = data.raw)
model.width = glm(spine ~ width, family = quasipoisson(link = log), data = data.raw)
model.sat = glm(spine ~ sat, family = quasipoisson(link = log), data = data.raw)

model.weight.weightcolor = glm(spine ~ weight + weight:color, data = data.raw, family = quasipoisson(link = log))
model.color.weight = glm(spine ~ color + weight, data = data.raw, family = quasipoisson(link = log))
model.color.width = glm(spine ~ color + width, data = data.raw, family = quasipoisson(link = log))
model.color.y = glm(spine ~ color + y, data = data.raw, family = quasipoisson(link = log))
model.color.sat = glm(spine ~ color + sat, data = data.raw, family = quasipoisson(link = log))

In [8]:
summary(model.color)


Call:
glm(formula = spine ~ color, family = quasipoisson(link = log), 
    data = data.raw)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2740  -0.2725   0.1347   0.3656   1.2378  

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   0.2877     0.1209   2.380   0.0184 *  
color2        0.5922     0.1250   4.737 4.58e-06 ***
color3        0.7321     0.1286   5.695 5.35e-08 ***
color4        0.7644     0.1354   5.648 6.76e-08 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for quasipoisson family taken to be 0.2337508)

    Null deviance: 56.157  on 172  degrees of freedom
Residual deviance: 45.604  on 169  degrees of freedom
AIC: NA

Number of Fisher Scoring iterations: 4


In [9]:
model.color.weight = glm(spine ~ color + weight, family = quasipoisson(link = log), data = data.raw)
summary(model.color.weight)


Call:
glm(formula = spine ~ color + weight, family = quasipoisson(link = log), 
    data = data.raw)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2336  -0.2930   0.1252   0.3479   1.2362  

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.41312    0.16407   2.518   0.0127 *  
color2       0.58754    0.12528   4.690 5.63e-06 ***
color3       0.71621    0.12952   5.530 1.21e-07 ***
color4       0.74259    0.13692   5.424 2.01e-07 ***
weight      -0.04777    0.04223  -1.131   0.2596    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for quasipoisson family taken to be 0.2344313)

    Null deviance: 56.157  on 172  degrees of freedom
Residual deviance: 45.302  on 168  degrees of freedom
AIC: NA

Number of Fisher Scoring iterations: 4


In [11]:
require(formula.tools)
require(boot)
options(warn=-1)

crossValidation <- function(mod) {
    set.seed(12345)
    rawError = signif(cv.glm(data.raw[, 2:7], mod, K = 10)$delta[1], 3)
    adjustedError = signif(cv.glm(data.raw[, 2:7], mod, K = 10)$delta[2], 3)
    c(rawCVError=rawError, adjustedCVError=adjustedError)
}

models = list(color = model.color, 
              all_variables = model.all_variables,
              color.weight = model.color.weight,
              color.width = model.color.width,
              color.y = model.color.y,
              color.sat = model.color.sat,
              weight.weightcolor = model.weight.weightcolor
             )

print(sapply(models, crossValidation, simplify = FALSE))

$color
     rawCVError adjustedCVError 
          0.572           0.568 

$all_variables
     rawCVError adjustedCVError 
          0.600           0.588 

$color.weight
     rawCVError adjustedCVError 
          0.585           0.577 

$color.width
     rawCVError adjustedCVError 
          0.589           0.577 

$color.y
     rawCVError adjustedCVError 
          0.573           0.571 

$color.sat
     rawCVError adjustedCVError 
          0.584           0.572 

$weight.weightcolor
     rawCVError adjustedCVError 
          0.585           0.581 

